In [1]:
from transformers import AutoConfig, AutoTokenizer, AutoModel
from data.acronymDataset import AcronymDataset

/opt/anaconda3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
model_name = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=config.max_position_embeddings)
pre_trained_model = AutoModel.from_pretrained(model_name, config=config)

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Test with the data

In [3]:
file_path = 'data/acronym_data.txt'
dataset = AcronymDataset(file_path=file_path, tokenizer=tokenizer)
data = dataset.data

In [4]:
dataset.preprocss_dataset()
data_tokens = dataset.preprocessed_dataset

Map:   0%|          | 0/74000 [00:00<?, ? examples/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [6]:
tokenizer.decode(data_tokens[1]['input_ids'])
# tokenizer.batch_decode(data_tokens[0]['input_ids'])

'[CLS] she is now bleeding quite heavily. ultrasound this morning demonstrated a missed < start > ab < end > consistent with a 6 week pregnancy with blood clots in the uterine cavity, as well as continued bleeding from the cervical os. this is consistent with an incomplete ab. the patient presents now for a suction d & c. medical history is negative. surgical history is negative. current medications : include prenatal vitamins. [SEP] she is now bleeding quite heavily. ultrasound this morning demonstrated a missed ab consistent with a 6 week pregnancy with blood clots in the uterine cavity, as well as continued bleeding from the cervical os. this is consistent with an incomplete < start > abortion < end > the patient presents now for a suction d & c. medical history is negative. surgical history is negative. current medications : include prenatal vitamins. [SEP]'

In [6]:
from models.multiHeadModel import MultiHeadModel
from models.heads import ClassificationHead

In [7]:
in_features = config.hidden_size
two_labels_head = ClassificationHead(in_features=in_features, out_features=2)
four_labels_head = ClassificationHead(in_features=in_features, out_features=4)

classifiers = [two_labels_head, four_labels_head]


In [8]:
multi_head_model = MultiHeadModel(pre_trained_model, classifiers)

In [10]:

output = multi_head_model(data_tokens[1], 0)
output

tensor([[0.5031, 0.5357]], grad_fn=<SigmoidBackward0>)

In [11]:
output = multi_head_model(data_tokens[1], 1)
output

tensor([[0.4523, 0.3779, 0.5768, 0.3946]], grad_fn=<SigmoidBackward0>)